In [127]:
import pandas as pd

data = pd.read_csv('data/22-23/player_data.csv')

In [128]:
def add_team_performance(data, window_size=38):

    x_home_goals = data[data['was_home']].groupby(
        ['fixture', 'team_h'], as_index=False)[['expected_goals']].sum()
    x_away_goals = data[~data['was_home']].groupby(
        ['fixture', 'team_a'], as_index=False)[['expected_goals']].sum()

    xG_table = x_away_goals.merge(
        x_home_goals, on='fixture', suffixes=['_a', '_h'])

    x_home_goals = xG_table[
        ['fixture', 'team_h', 'expected_goals_h', 'expected_goals_a']].rename(
        columns={
            'team_h': 'team',
            'expected_goals_h': 'xG', 'expected_goals_a': 'xGA'})

    x_away_goals = xG_table[
        ['fixture', 'team_a', 'expected_goals_a', 'expected_goals_h']].rename(
            columns={
                'team_a': 'team',
                'expected_goals_a': 'xG', 'expected_goals_h': 'xGA'})

    xG_table = pd.concat([x_home_goals, x_away_goals])

    xG_cumulative = xG_table.groupby(['team', 'fixture']).sum().groupby(
        level=0).rolling(window_size, min_periods=1).mean(
        ).droplevel(0).reset_index()

    xG_cumulative = xG_cumulative[['fixture']].join(xG_cumulative.sort_values(
        'fixture').groupby('team').shift(1)[['xG', 'xGA']]).dropna(how='any')

    return data.merge(xG_cumulative.drop_duplicates('fixture').merge(
        xG_cumulative.drop_duplicates('fixture', keep='last'),
        on='fixture', suffixes=['_h', '_a']), on='fixture')

In [129]:
data = add_team_performance(data)

In [117]:





x_home_goals = data[data['was_home']].groupby(['fixture','team_h'], as_index=False)[['expected_goals']].sum()
x_away_goals = data[~data['was_home']].groupby(['fixture','team_a'], as_index=False)[['expected_goals']].sum()


xG_table = x_away_goals.merge(x_home_goals, on='fixture', suffixes=['_a', '_h'])

x_home_goals = xG_table[['fixture', 'team_h', 'expected_goals_h', 'expected_goals_a']].rename(
        columns={'team_h':'team', 'expected_goals_h': 'xG', 'expected_goals_a': 'xGA'})

x_away_goals = xG_table[
    ['fixture', 'team_a', 'expected_goals_a', 'expected_goals_h']].rename(
        columns={'team_a':'team', 'expected_goals_a': 'xG', 'expected_goals_h': 'xGA'})

xG_table = pd.concat([x_home_goals, x_away_goals])

xG_cumulative = xG_table.groupby(['team', 'fixture']).sum().groupby(level=0).rolling(38, min_periods=1).mean().droplevel(0).reset_index()

xG_cumulative = xG_cumulative[['fixture']].join(xG_cumulative.sort_values('fixture').groupby('team').shift(1)[['xG', 'xGA']]).dropna(how='any')

data = data.merge(xG_cumulative.drop_duplicates('fixture').merge(xG_cumulative.drop_duplicates('fixture', keep='last'), on='fixture', suffixes=['_h', '_a']), on='fixture')
